# El Yapımı Standardizer

🧑🏻‍🏫 Bu challenge’da *kendi* StandardScaler’ımızı oluşturacağız. Neden mi? Güzel soru!

🎯 Bu egzersizin amaçları şunlardır:
- `stateless transformers` ile `stateful transformers` arasındaki farkı anlamak
- `FeatureUnion` ile çalışmak

## (1) 📚 Stateless Transformer vs. Stateful Transformer

🔢 Aşağıdaki training set’i ve aşağıdaki test set’i göz önünde bulundurun...

In [1]:
import numpy as np
import pandas as pd

X_train = pd.DataFrame({
    'A': {0: 1, 1: 2, 2: 3},
    'B': {0: 4, 1: 5, 2: 6},
    'C': {0: 7, 1: 8, 2: 9}
})

print("This is the training dataset:")
display(X_train)

X_test = pd.DataFrame({
    'A': {0: 1, 1: 2, 2: 3},
    'B': {0: 2, 1: 3, 2: 4},
    'C': {0: 3, 1: 4, 2: 10}
})

print("This is the test dataset:")
display(X_test)

This is the training dataset:


,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


This is the test dataset:


,A,B,C
0,1,2,3
1,2,3,4
2,3,4,10


🛠 ...ve aşağıdaki FeatureUnion’ı düşünün; bu yapı:
- feature’ları ölçekler
- diğer (ölçeklenmemiş) feature’ların ortalaması olan yeni bir feature oluşturur

In [2]:
from sklearn import set_config; set_config(display='diagram')
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import make_pipeline, make_union

standard_scaler = StandardScaler()
feature_averager = FunctionTransformer(lambda df: pd.DataFrame(1/3 * (df["A"] + df["B"] + df["C"])))

pipeline = make_union(standard_scaler, feature_averager)
pipeline

FeatureUnion(transformer_list=[('standardscaler', StandardScaler()),
                               ('functiontransformer',
                                FunctionTransformer(func=<function <lambda> at 0x1239b99e0>))])

Hadi:
- pipeline’ı training set üzerinde fit edelim
- hem training set’i hem de test set’i transform edelim

In [3]:
pipeline.fit(X_train)

FeatureUnion(transformer_list=[('standardscaler', StandardScaler()),
                               ('functiontransformer',
                                FunctionTransformer(func=<function <lambda> at 0x1239b99e0>))])

In [4]:
X_train_transformed = pd.DataFrame(pipeline.transform(X_train))
X_train_transformed

,0,1,2,3
0,-1.224745,-1.224745,-1.224745,4.0
1,0.000000,0.000000,0.000000,5.0
2,1.224745,1.224745,1.224745,6.0


In [5]:
X_test_transformed = pd.DataFrame(pipeline.transform(X_test))
X_test_transformed

,0,1,2,3
0,-1.224745,-3.674235,-6.123724,2.000000
1,0.000000,-2.449490,-4.898979,3.000000
2,1.224745,-1.224745,2.449490,5.666667


👨🏻‍🏫 `StandardScaler` ile `FunctionTransformer`’ın temelde ne kadar farklı olduğuna dikkat edin ❗️

Pipeline’ı fit edip hem training set’i hem de test set’i transform ettiğimizde:

* **`FunctionTransformer (feature_averager)`**:
    * *.fit()* sırasında hiçbir şey “öğrenmedi”
    * yalnızca **stateless bir dönüşüm** uyguladı: $ \large (X_1, X_2, X_3) \rightarrow \frac{(X_1 + X_2 + X_3)}{3}$


* **`StandardScaler`**:
    * *.fit()* sırasında $\mu_{\color{blue}{train}}$ ve $\sigma_{\color{blue}{train}}$ değerlerini “öğrendi”
    * bu öğrenilen değerleri kullanarak hem training set’te hem de test set’te **stateful bir dönüşüm** gerçekleştirdi:
        * $ \large X_{\color{blue}{train-scaled}} =  \frac{X_{\color{blue}{train}} -\mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}$
        * $ \large X_{\color{red}{test-scaled}} =  \frac{X_{\color{red}{test}} -\mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}$

## (2) 💻 Kendi stateful transformer’ınızı oluşturun

🤔 Peki ya kendi **stateful custom transformer**’ımızı kodlamak istersek?

💪 Kendi class’ımızı yazabiliriz!

# El Yapımı Standardizer

### (2.1) 💻 Custom Standardizer

❓ **Sorular: Kendi class’ınızı kodlama** ❓

1. Kendi `CustomStandardizer` class’ınızı yazın  
    * Scikit Learn’deki `StandardScaler` ile **birebir aynı davranışı** göstermelidir. Yani:
        * $\mu_{\color{blue}{train}}$ ve $\sigma_{\color{blue}{train}}$ değerlerini hesaplayan (“öğrenen”) bir `.fit()` metodu
        * ve bir `.transform()` metodu olmalıdır.

2. Bu class’ı `X_train` üzerinde fit edin

3. Hem `X_train` hem de `X_test` üzerinde transform uygulayın

4. Her şeyi doğru yaptığınızdan emin olmak için `CustomStandardizer`’ınızı Scikit Learn’deki `StandardScaler` ile karşılaştırın!

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class CustomStandardizer(TransformerMixin, BaseEstimator):
    
    def __init__(self):
        self.mean_ = None
        self.std_ = None
    
    def fit(self, X, y=None):
        X = np.array(X)
        self.mean_ = X.mean(axis=0)
        self.std_ = X.std(axis=0)
        return self
    
    def transform(self, X, y=None):
        X = np.array(X)
        X_scaled = (X - self.mean_) / self.std_
        return X_scaled

🧪 **Kodunu Test Et**

In [11]:
from nbresult import ChallengeResult

tmp = CustomStandardizer()
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult(
    'standardizer',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/dilay/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/dilay/Desktop/S17D1-S-Data-hand-made-standardizer/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_standardizer.py::TestStandardizer::test_solution PASSED             [100%]

============================== 1 passed in 0.10s ===============================


💯 You can commit your code:

git add tests/standardizer.pickle

git commit -m 'Completed standardizer step'

git push origin master



<details>
<summary>💡 <i>İpuçları</i> (🧪 eğer yukarıdaki testler yalnızca küçük bir farkla başarısız oluyorsa)</summary>

* `np.std()` ile `pd.std()` metotları arasında **küçük bir fark** olduğuna dikkat edin!
    
* Bu [Stack Overflow gönderisi](https://stackoverflow.com/questions/44220290/sklearn-standardscaler-result-different-to-manual-result) yardımcı olabilir 😉
      
</details>

<details>
<summary>💡 <i>Çözüm</i></summary>

Eğer `pd.std` metodunu kullanıyorsanız, scikit-learn’ün `StandardScaler` davranışını birebir taklit edebilmek için `ddof=0` parametresini de kullanmanız gerekir.      
</details>

### (2.2) 💻 Inverse Transform

❓ **Question (Inverse Transform)** ❓

_StandardScaler_ from Scikit Learn has a [`.inverse_transform()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler.inverse_transform) method that helps you revert back to the unscaled dataset.

1. Go back to your `CustomStandardizer` class and implement your own `.inverse_transform()` method.

2. Try it on your scaled training set and your scaled test set.

In [23]:

from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class CustomStandardizer(TransformerMixin, BaseEstimator):
    
    def __init__(self):
        self.mean_ = None
        self.std_ = None
    
    def fit(self, X, y=None):
        X = np.array(X)
        self.mean_ = X.mean(axis=0)
        self.std_ = X.std(axis=0)
        return self
    
    def transform(self, X, y=None):
        X = np.array(X)
        X_scaled = (X - self.mean_) / self.std_
        return X_scaled
    
    def inverse_transform(self, X_scaled):
        X_scaled = np.array(X_scaled)
        X = X_scaled * self.std_ + self.mean_
        return X


custom_standardizer = CustomStandardizer()
custom_standardizer.fit(X_train)


X_train_scaled = custom_standardizer.transform(X_train)
X_test_scaled = custom_standardizer.transform(X_test)

In [24]:

X_train_inverse_transformed = custom_standardizer.inverse_transform(X_train_scaled)
X_test_inverse_transformed = custom_standardizer.inverse_transform(X_test_scaled)

🧪 **Kodunu Test Et**

In [25]:
assert np.allclose(X_train_inverse_transformed, X_train)
assert np.allclose(X_test_inverse_transformed, X_test)

### (2.3) 💻 Özel (custom) pipeline tamamlandı!

💪 Scikit-Learn’in `StandardScaler`’ını birebir kopyalamayı başardık.

🌶 Şimdi biraz daha heyecan katalım!

❓ **Soru: Önceki `CustomStandardizer` özel transformer’ını bir shrink factor ile geliştirin** ❓

`CustomStandardizer(shrink_factor = 1)` class’ı, ölçeklemeyi daha güçlü hale getirmek için **ek bir argüman** almalıdır. Bu durumda ölçekleme, $\sigma_{\color{blue}{train}}$ ile **orantılı** olacaktır 👇:

- $ \large X_{\color{blue}{train-scaled}} =  \left(\frac{X_{\color{blue}{train}} - \mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}\right) \times \frac{1}{shrink\ factor}$
- $ \large X_{\color{red}{test-scaled}} =  \left(\frac{X_{\color{red}{test}} - \mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}\right) \times \frac{1}{shrink\ factor}$

In [26]:
###################################
# Custom Standardizer             #
###################################



class CustomStandardizer(TransformerMixin, BaseEstimator):

    
    def __init__(self, shrink_factor=1):
        self.shrink_factor = shrink_factor
        self.mean_ = None
        self.std_ = None

    def fit(self, X, y=None):
        '''
        Stores what needs to be stored as instance attributes.
        Returns "self" to allow chaining fit and transform.
        '''
        X = np.array(X)
        self.mean_ = X.mean(axis=0)
        self.std_ = X.std(axis=0)
        return self

    
    def transform(self, X, y=None):
        X = np.array(X)
        X_scaled = ((X - self.mean_) / self.std_) * (1 / self.shrink_factor)
        return X_scaled

    
    def inverse_transform(self, X_scaled, y=None):
        X_scaled = np.array(X_scaled)
        X = (X_scaled * self.shrink_factor) * self.std_ + self.mean_
        return X

🧪 **Yeni `CustomStandardizer` özel transformer’ınızı test edin** (`shrink_factor = 2` ile):

- `X_train` üzerinde fit edin
- Hem `X_train` hem de `X_test` üzerinde transform uygulayın
- Dönüştürülmüş DataFrame’leri `X_train_transformed` ve `X_test_transformed` değişkenlerinde saklayın

# El Yapımı Standardizer

In [32]:
custom_standardizer = CustomStandardizer(shrink_factor=2)
custom_standardizer.fit(X_train)

X_train_transformed = custom_standardizer.transform(X_train)
X_test_transformed = custom_standardizer.transform(X_test)
display(X_train_transformed)
display(X_test_transformed)


array([[-0.61237244, -0.61237244, -0.61237244],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.61237244,  0.61237244,  0.61237244]])

array([[-0.61237244, -1.83711731, -3.06186218],
       [ 0.        , -1.22474487, -2.44948974],
       [ 0.61237244, -0.61237244,  1.22474487]])

🧪 **Kodunu Test Et**

In [30]:
from nbresult import ChallengeResult

tmp = CustomStandardizer(shrink_factor=2).fit(X_train)
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult(
    'new_standardizer',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/dilay/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/dilay/Desktop/S17D1-S-Data-hand-made-standardizer/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_new_standardizer.py::TestNewStandardizer::test_solution PASSED      [100%]

============================== 1 passed in 0.12s ===============================


💯 You can commit your code:

git add tests/new_standardizer.pickle

git commit -m 'Completed new_standardizer step'

git push origin master



In [ ]:
# Aşağıdaki hücreleri çalıştırarak doğru dönüşümleri elde ettiğinizden emin olun
# truth_train = np.array([
#     [-0.612372, -0.612372, -0.612372],
#     [0.000000, 0.000000, 0.000000],
#     [0.612372, 0.612372, 0.612372]
# ])
# truth_test = np.array([
#     [-0.612372, -1.837117, -3.061862],
#     [ 0.        , -1.224745, -2.449490],
#     [ 0.612372, -0.612372,  1.224745]])

In [ ]:
# # Assert’ler
# np.allclose(X_train_transformed, truth_train)

In [ ]:
# # Assert - Test
# np.allclose(X_test_transformed, truth_test)

❓ **Soru: Önceki `FeatureAverager` özel transformer’ını “tweak” edin** ❓

Bu değiştirilmiş `FeatureAverager()` class’ı:
- hâlâ üç farklı feature’ın ortalamasını hesaplar...
- ...ve ardından sonucu her satır için maksimum değere böler

_Not_: Bu işlemi yorumlamaya çalışmayın. Gerçek bir anlamı yok; amacımız yalnızca özel bir class kodlama becerilerimizi pratik etmek.

$$(X_1, X_2, X_3) \rightarrow \frac{mean(X_1 + X_2 + X_3)}{max(X_1, X_2, X_3)}$$

_Not_: Burada **stateful** mı yoksa **stateless** bir dönüşümle mi uğraştığımızı dikkatlice düşünün (yani, *stateful* `StandardScaler`’da yaptığımız gibi dönüşümden önce tüm training verisi hakkında bilgi öğrenmemiz gerekiyor mu, yoksa yukarıdaki *stateless* `feature_averager` gibi mi çalışıyor?). İlki söz konusuysa `.fit()` için bir şeyler yazmamız gerekir; değilse gerekmez!

In [33]:
###################################
# Feature Averager                #
###################################

class FeatureAverager(TransformerMixin, BaseEstimator):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        '''
        Gerekirse bu metot, bilgileri instance attribute’ları olarak saklar.
        "self"  return eder .
        '''
        return self

    def transform(self, X, y=None):
        X = np.array(X)
        
        mean_values = X.mean(axis=1)
        
        max_values = X.max(axis=1)
        
        result = mean_values / max_values
        
        return result.reshape(-1, 1)

🧪 **Test you `FeatureAverager` custom transformer** by fitting on `X_train`  and transforming both `X_train` and `X_test`

In [35]:
feature_averager = FeatureAverager()
feature_averager.fit(X_train)

X_train_averaged = feature_averager.transform(X_train)
X_test_averaged = feature_averager.transform(X_test)


print(f"\n X_train_averaged shape: {X_train_averaged.shape}")
print(f"X_train_averaged:\n{X_train_averaged}")
print(f"\n X_test_averaged shape: {X_test_averaged.shape}")
print(f"X_test_averaged:\n{X_test_averaged}")


 X_train_averaged shape: (3, 1)
X_train_averaged:
[[0.57142857]
 [0.625     ]
 [0.66666667]]

 X_test_averaged shape: (3, 1)
X_test_averaged:
[[0.66666667]
 [0.75      ]
 [0.56666667]]


In [36]:
from nbresult import ChallengeResult

tmp = FeatureAverager()
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult(
    'feature_averager',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/dilay/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/dilay/Desktop/S17D1-S-Data-hand-made-standardizer/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_feature_averager.py::TestFeatureAverager::test_solution PASSED      [100%]

============================== 1 passed in 0.12s ===============================


💯 You can commit your code:

git add tests/feature_averager.pickle

git commit -m 'Completed feature_averager step'

git push origin master



❓ **Soru (Feature Union)** ❓

1. Hem `CustomStandardizer` hem de `FeatureAverager` kullanarak bir `FeatureUnion` pipeline oluşturun ve bunu `pipeline` adlı bir değişkende saklayın.

2. Pipeline’ı `X_train` üzerinde fit edin ve hem `X_train` hem de `X_test` üzerinde transform uygulayın (`shrink_factor = 3`).

3. Bu challenge’ın son testini geçtiğinizden emin olun.

In [37]:
from sklearn.pipeline import FeatureUnion

pipeline = FeatureUnion([
    ('standardizer', CustomStandardizer(shrink_factor=3)),
    ('averager', FeatureAverager())
])



In [38]:
pipeline.fit(X_train)

X_train_transformed = pipeline.transform(X_train)
X_test_transformed = pipeline.transform(X_test)

print(f"\n X_train_transformed shape: {X_train_transformed.shape}")
print(f"X_train_transformed:\n{X_train_transformed}")
print(f"\n📊X_test_transformed shape: {X_test_transformed.shape}")
print(f"X_test_transformed:\n{X_test_transformed}")


 X_train_transformed shape: (3, 4)
X_train_transformed:
[[-0.40824829 -0.40824829 -0.40824829  0.57142857]
 [ 0.          0.          0.          0.625     ]
 [ 0.40824829  0.40824829  0.40824829  0.66666667]]

📊X_test_transformed shape: (3, 4)
X_test_transformed:
[[-0.40824829 -1.22474487 -2.04124145  0.66666667]
 [ 0.         -0.81649658 -1.63299316  0.75      ]
 [ 0.40824829 -0.40824829  0.81649658  0.56666667]]


In [39]:
from nbresult import ChallengeResult

tmp = pipeline
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))



result = ChallengeResult(
    'feature_union_custom_transformers',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/dilay/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/dilay/Desktop/S17D1-S-Data-hand-made-standardizer/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_feature_union_custom_transformers.py::TestFeatureUnionCustomTransformers::test_solution PASSED [100%]

============================== 1 passed in 0.16s ===============================


💯 You can commit your code:

git add tests/feature_union_custom_transformers.pickle

git commit -m 'Completed feature_union_custom_transformers step'

git push origin master



🏁 Tebrikler! Kendi Transformer’ınızı nasıl oluşturacağınızı keşfettiniz!

💾 Notebook’unuzu `git add / commit / push` yapmayı unutmayın...

🚀 ... ve bir sonraki challenge’a geçin!